In [2]:
# !wget https://raw.githubusercontent.com/mesolitica/malaysian-dataset/master/llm-benchmark/tatabahasabm.tripod.com-bm-kertas-1/tatabahasabm.tripod.com-bm-kertas1.json

In [37]:
import json

with open('tatabahasabm.tripod.com-bm-kertas1.json') as fopen:
    data = json.load(fopen)

In [7]:
instructions = []
for k, v in data.items():
    for r in v['quizzes'].values():
        if r['no'] in v['answers']:
            c = '\n'.join(r['choice'])
            s = f"{r['objective']}\n{r['context']}\n\n{r['question']}\n{c}\nJawapan: {v['answers'][r['no']]}"
            instructions.append(s)

In [8]:
print(instructions[0])

Baca petikan di bawah, kemudian jawab soalan-soalan berikutnya
Gopal : Eh, awak ingat lagi tak tentang berita manusia ditelan ular sawa tempoh hari!  
Samat : Mestilah Sampai sekarang saya masih terbayang-bayang gambar yang tersiar 
dalam akhbar tu. 
Eee............ngerinya! 
Chan : Tapi yang peliknya, lepas kejadian tu tak putus-putus keluar cerita tentang ular sawa 
dalam akhbar. Ular sawa telan ayamlah, telan kambinglah, dan macam -macam lagi 
Samat : Apa yang peliknya, Chan Memanglah berita sensasi macam tu disukai oleh orang 
ramai Jadi wartawan pun jalankanlah tugas mereka untuk mencari cerita -cerita sedemikian! 
Gopal : Agaknya tak lama lagi ada pula berita tentang ular sawa telan gajah. Haa. masa  tu 
barulah heboh satu dunia! Adakah awak tahu ular sawa merupakan sejenis haiwan yang 
dilindungi? 
Samat : Tahu. baru semalam Cikgu Lina beritahu 
Gopal : Yalah, selama ni saya tak tahu pun yang kita boleh didakwa jika mencedera atau 
membunuh ular. 
Chan : Hukumannya berat juga Ki

In [9]:
import os
import openai

openai.api_type = "azure"
openai.api_base = "https://husein-ai2-aiservices.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = ''

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [29]:
!mkdir tatabahasabm.tripod.com-bm-kertas1
# !rm tatabahasabm.tripod.com-bm-kertas1/*

mkdir: cannot create directory ‘tatabahasabm.tripod.com-bm-kertas1’: File exists


In [31]:
from tqdm import tqdm
import os

for i in tqdm(range(0, len(instructions), 1)):
    filename = f'tatabahasabm.tripod.com-bm-kertas1/{i}.json'
    if os.path.exists(filename):
        continue
    try:
        message_text = [
            {"role":"user","content": f"{instructions[i]}\n\ngenerate similar questions included answers like above."},
        ]
        completion = openai.ChatCompletion.create(
          engine="gpt-4",
          messages = message_text,
          temperature=1.0,
          max_tokens=1024,
          top_p=0.95,
          frequency_penalty=0,
          presence_penalty=0,
          stop=None
        )
        with open(filename, 'w') as fopen:
            json.dump(completion.choices[0]['message']['content'], fopen)
    except Exception as e:
        print(e)

 25%|██████████▊                                | 25/99 [00:11<00:35,  2.10it/s]

The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766


100%|███████████████████████████████████████████| 99/99 [15:28<00:00,  9.38s/it]


In [38]:
instructions = []
for k, v in data.items():
    for r in v['quizzes'].values():
        if r['no'] in v['answers']:
            c = '\n'.join(r['choice'])
            instructions.append((r, v['answers'][r['no']]))

In [39]:
instructions[0]

({'objective': 'Baca petikan di bawah, kemudian jawab soalan-soalan berikutnya',
  'context': 'Gopal : Eh, awak ingat lagi tak tentang berita manusia ditelan ular sawa tempoh hari!  \nSamat : Mestilah Sampai sekarang saya masih terbayang-bayang gambar yang tersiar \ndalam akhbar tu. \nEee............ngerinya! \nChan : Tapi yang peliknya, lepas kejadian tu tak putus-putus keluar cerita tentang ular sawa \ndalam akhbar. Ular sawa telan ayamlah, telan kambinglah, dan macam -macam lagi \nSamat : Apa yang peliknya, Chan Memanglah berita sensasi macam tu disukai oleh orang \nramai Jadi wartawan pun jalankanlah tugas mereka untuk mencari cerita -cerita sedemikian! \nGopal : Agaknya tak lama lagi ada pula berita tentang ular sawa telan gajah. Haa. masa  tu \nbarulah heboh satu dunia! Adakah awak tahu ular sawa merupakan sejenis haiwan yang \ndilindungi? \nSamat : Tahu. baru semalam Cikgu Lina beritahu \nGopal : Yalah, selama ni saya tak tahu pun yang kita boleh didakwa jika mencedera atau \nme

In [61]:
augmentation = []
for i in tqdm(range(0, len(instructions), 1)):
    filename = f'tatabahasabm.tripod.com-bm-kertas1/{i}.json'
    try:
        with open(filename) as fopen:
            data = json.load(fopen)
        splitted = data.split('\n\n')
        for s in splitted:
            a = s.split('Jawapan:')[1].strip()
            augmentation.append({
                'objective': instructions[i][0]['objective'],
                'context': instructions[i][0]['context'],
                'question': s.split('Jawapan:')[0].strip(),
                'answer': a,
                
            })
    except:
        pass

100%|████████████████████████████████████████| 99/99 [00:00<00:00, 12795.79it/s]


In [62]:
len(augmentation)

439

In [78]:
augmentation[100]

{'objective': 'Pilih jawapan yang paling sesuai untuk diisi ke tempat kosong dalam ayat-ayat di bawah.',
 'context': '',
 'question': '1. Kesungguhannya dalam belajar _________________ apabila dia mendapat keputusan cemerlang dalam peperiksaan.\nA. terbukti\nB. nyata\nC. kelihatan\nD. bermula',
 'answer': 'A'}

In [79]:
with open('synthetic-tatabahasabm.tripod.com-bm-kertas1.jsonl', 'w') as fopen:
    for a in augmentation:
        fopen.write(f'{json.dumps(a)}\n')